Luke Lawn, Oriana McDonough & Ryan Pike (Group 14)
Reviewing Instructor: Angela Ramnarine-Rieks

Project Overview: Our project will take the top listened artists from a user’s Spotify account and return concerts and venues where that artist is scheduled to perform. This will save people time and allow them to easily see the artists they love listening to performing live, where they can then purchase concert tickets if they choose to. Our goal is to provide users a new way to preview which shows and events they may want to attend based on their personalized listening history.

Inputs: Spotify account username

Outputs: Concert tickets options (price, date, time) and venue information (location, contact information) for top listened artists

Algorithm: 
1 - Import Spotify API and TicketMaster API
2 - User inputs their Spotify account username
3 - Create a list of the user’s top 10 listened artists
4 - For each artist in the list, find upcoming concerts and venue information
5 - Print the above

In [ ]:
!pip install simplejson
!pip install spotipy
!pip install ticketpy


In [30]:
import sys
import spotipy
import spotipy.util as util
username = input('Input an account name:')
scope = 'user-library-read'
token = util.prompt_for_user_token(username,scope,client_id='69d411268d5d47c7adc4dc6868928548',client_secret='db95e59f1fa242d28a897684312a85e6',redirect_uri='https://accounts.spotify.com/api/token')

if token:
    sp = spotipy.Spotify(auth=token)
    results = sp.current_user_saved_tracks()
    for item in results['items']:
        track = item['track']
        print (track['name'] + ' - ' + track['artists'][0]['name'])
else:
    print("Can't get token for", username)

Input an account name:yes
Pick It Up (feat. A$AP Rocky) - Famous Dex
Juice - Yo Gotti
Undefeated (feat. 21 Savage) - A Boogie Wit da Hoodie
Real Thing (feat. Future) - Tory Lanez
Lil Story (feat. ScHoolboy Q) - Gucci Mane
Money Make Ya Handsome - Gucci Mane
FYBR (First Year Being Rich) - A$AP Mob
Immortal - J. Cole
It's A Vibe - 2 Chainz
Hate It Or Love It - The Game
Plain Jane - A$AP Ferg
Big Fish - Vince Staples
coordinate - Travis Scott
ELEMENT. - Kendrick Lamar
Patty Cake - Kodak Black
Dark Queen - Lil Uzi Vert
Neon Guts (feat. Pharrell Williams) - Lil Uzi Vert
XO TOUR Llif3 - Lil Uzi Vert
I Just Wanna Party - YG
Watching Movies - Mac Miller


In [7]:
import pprint
import sys

import spotipy
import spotipy.util as util
import simplejson as json
username = input("Would you like to link your spotify account? ")


if len(sys.argv) > 1:
    username = sys.argv[1]
else:
    print("Usage: %s username" % (sys.argv[0],))
    sys.exit()
scope = 'user-top-read'
token = util.prompt_for_user_token(username,scope,client_id='69d411268d5d47c7adc4dc6868928548',client_secret='db95e59f1fa242d28a897684312a85e6',redirect_uri='https://accounts.spotify.com/api/token')

if token:
    sp = spotipy.Spotify(auth=token)
    sp.trace = False
    ranges = ['medium_term']
    for range in ranges:
        print("range:", range)
        results = sp.current_user_top_artists(time_range=range, limit=10)
        for i, item in enumerate(results['items']):
            print(i, item['name'])
else:
    print("Can't get token for", username)
    


Would you like to link your spotify account? ye
range: medium_term
0 Lil Uzi Vert
1 A$AP Mob
2 Kodak Black
3 Travis Scott
4 Gucci Mane
5 Meek Mill
6 Sublime
7 21 Savage
8 Drake
9 2 Chainz


In [17]:
import ticketpy

tm_client = ticketpy.ApiClient('QQJB3jgFw4OyEe77Y7OGGsFGfqQ0f0hM')

pages = tm_client.events.find(
    artist_name='Lil Uzi Vert',
    state_code='GA',
).limit()

for event in pages:
    print(event)

Event:            BREAKING THE CHAINS TOUR W/ THEOCRACY & FANGHORN
Venues:           [<ticketpy.model.Venue object at 0x000002195E3CC550>]
Start date:       2017-12-03
Start time:       18:00:00
Price ranges:     []
Status:           offsale
Classifications:  [<ticketpy.model.EventClassification object at 0x000002195E3CC390>]

Event:            In the Atlanta Room:  MOSES MO / BROOKS HUBBARD BAND / ROB AGOCS
Venues:           [<ticketpy.model.Venue object at 0x000002195E3CC5C0>]
Start date:       2017-12-03
Start time:       18:30:00
Price ranges:     []
Status:           offsale
Classifications:  [<ticketpy.model.EventClassification object at 0x000002195E3CC518>]

Event:            GWAR - The Blood of Gods Tour
Venues:           [<ticketpy.model.Venue object at 0x000002195E3CC7B8>]
Start date:       2017-12-03
Start time:       18:30:00
Price ranges:     [{'min': 20.0, 'max': 20.0}]
Status:           offsale
Classifications:  [<ticketpy.model.EventClassification object at 0x000002195E

In [29]:
import ticketpy
import requests
import json


url = 'https://app.ticketmaster.com/discovery/v2/events.json?countryCode=US&apikey=QQJB3jgFw4OyEe77Y7OGGsFGfqQ0f0hM'
response = requests.get(url)
events = response.json()
artists = events['name']

print(events)

{'_embedded': {'events': [{'name': 'Seattle Seahawks vs. Philadelphia Eagles', 'type': 'event', 'id': 'vvG1HZfIKQAMqF', 'test': False, 'url': 'http://www.ticketmaster.com/seattle-seahawks-vs-philadelphia-eagles-seattle-washington-12-03-2017/event/0F005284DA177748', 'locale': 'en-us', 'images': [{'ratio': '4_3', 'url': 'https://s1.ticketm.net/dam/a/7ef/0e94936b-1d04-4448-b1d3-cc375102e7ef_549731_CUSTOM.jpg', 'width': 305, 'height': 225, 'fallback': False}, {'ratio': '16_9', 'url': 'https://s1.ticketm.net/dam/a/7ef/0e94936b-1d04-4448-b1d3-cc375102e7ef_549731_EVENT_DETAIL_PAGE_16_9.jpg', 'width': 205, 'height': 115, 'fallback': False}, {'ratio': '16_9', 'url': 'https://s1.ticketm.net/dam/a/7ef/0e94936b-1d04-4448-b1d3-cc375102e7ef_549731_TABLET_LANDSCAPE_16_9.jpg', 'width': 1024, 'height': 576, 'fallback': False}, {'ratio': '3_2', 'url': 'https://s1.ticketm.net/dam/a/7ef/0e94936b-1d04-4448-b1d3-cc375102e7ef_549731_TABLET_LANDSCAPE_3_2.jpg', 'width': 1024, 'height': 683, 'fallback': False},

In [19]:
tm_client = ticketpy.ApiClient("QQJB3jgFw4OyEe77Y7OGGsFGfqQ0f0hM")
attractions = tm_client.attractions.find(keyword="Justin Bieber").one()
for attr in attractions:
    print(attr)

Justin Bieber
An Evening with Justin Bieber
Justin Beiber's Girlfriend
